In [1]:
import sys
sys.path.append('../data')
from answer import dictionary #real_w
from wordle import rand_select ,word_compare,end_game
from scipy.stats import entropy
import pandas as pd
import random
import time
real_W = sorted(dictionary)
pos_W =  sorted(dictionary)

In [2]:
def word_compare_1d(target,w_lst):
    '''word compare word to all words'''
    re_lst=[]
    for word in w_lst:
        re_lst.append(word_compare(target,word))
    return re_lst

def gen_table(w_lst1,w_lst2):
    '''
    gen lst1*lst2 table
    
    '''
    word_table={}
    for w1 in w_lst1:
        word_table[w1] = word_compare_1d(w1,w_lst2)
    return word_table

In [3]:
def best_entropy(table,step=0):
    best = -1
    best_ws =''
    for idx,key in enumerate(table):
        entro_score = entropy(table[key].value_counts().values/len(table),base=2)
        if entro_score>best:
            best = entro_score
            best_w = key
    #print("best word: {}, score: {}".format(best_w,best))
    return best_w

In [14]:
def best_entropies(table,step=0):
    best_dic={}
    for idx,key in enumerate(table):
        entro_score = entropy(table[key].value_counts().values/len(table),base=2)
        best_dic[key]=entro_score
    return {k: v for k, v in sorted(best_dic.items(),reverse=True, key=lambda item: item[1])}

In [5]:
def sub_table(table, guess_w, row_idx, answer):
    sub_s = table[guess_w] 
    hint = word_compare(guess_w,answer)
    match_s = (sub_s[sub_s == hint])#matched rows in series
    cand = table.columns[match_s.index]# candidates in list
    re_table = table[cand].iloc[list(match_s.index)]
    return re_table.reset_index(drop=True)

In [6]:
def guess(table,row_idx,step=0):
    '''
    step 0 -> find best entropy within raw table
    step > find best entropy within filtered table
    n*m tabel 
    n -> pos_w (table.column)
    m -> real_w
    '''
    guess_w = best_entropy(table)
    re_table = sub_table(table,guess_w,row_idx,ans)
    
    return re_table
        

In [7]:
table =  pd.DataFrame(gen_table(real_W,real_W))

In [15]:
test = best_entropies(table)
test

{'raise': 5.877909690821481,
 'slate': 5.855775376955966,
 'crate': 5.834874004263522,
 'irate': 5.831396980440787,
 'trace': 5.830548713859443,
 'arise': 5.8209397008860035,
 'stare': 5.807280035160929,
 'snare': 5.770088860457005,
 'arose': 5.767796702404354,
 'least': 5.751645645679271,
 'alert': 5.745836550944613,
 'crane': 5.742781744697181,
 'stale': 5.738572671790489,
 'saner': 5.733713267350087,
 'alter': 5.713170728077568,
 'later': 5.7060889572947895,
 'react': 5.696353951200044,
 'leant': 5.684573501404134,
 'trade': 5.681560869582508,
 'learn': 5.6560743820483745,
 'cater': 5.646714898485081,
 'roast': 5.645193450061066,
 'aisle': 5.636827622829473,
 'trice': 5.634236581503792,
 'scare': 5.630489493703489,
 'parse': 5.629691003590093,
 'saute': 5.62020090356796,
 'heart': 5.614518956629291,
 'alone': 5.614035882733971,
 'store': 5.610465734674016,
 'alien': 5.60953378917969,
 'share': 5.608309347595908,
 'grate': 5.6040831484404645,
 'trail': 5.59121796632083,
 'siren': 5.5

In [16]:
start = time.time()
total_count=[]
turns = 2315 # 做n次
for turn in range(turns):
    condition = False
    ans = real_W[turn]
    t = table.copy()
    count = 0
    while(condition==False):
        gue = best_entropy(t)
        t = sub_table(t,gue,real_W,ans)
        #print(len(t))
        hint = word_compare(gue,ans)
        #print("hint: ",hint)
        count+=1
        condition = end_game(hint)
    total_count.append(count)
end = time.time()
print("average guess: ", sum(total_count)/turns)
print("tooks {} sec to run {} turns".format(end-start,turns))

average guess:  3.614254859611231
tooks 1187.8257689476013 sec to run 2315 turns


In [19]:
analysis = {}
for elem in set(total_count):
    analysis[str(elem)]= total_count.count(elem)

In [20]:
analysis

{'1': 1, '2': 131, '3': 978, '4': 928, '5': 217, '6': 49, '7': 9, '8': 2}

In [21]:
complete_rate = (2315 - 11)/2315
excelency = (1+131)/2315

In [23]:
print("complete_rate: {}, excelency: {}, fail_rate: {}".format(complete_rate,excelency,1-complete_rate))

complete_rate: 0.9952483801295896, excelency: 0.05701943844492441, fail_rate: 0.004751619870410395
